In [20]:
import pandas as pd
import numpy as np
import re
import csv
import pickle


import urllib.request
%matplotlib inline
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import RMSprop, Adam

from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Dropout, Conv1D, MaxPooling1D, Flatten
from tensorflow.keras.models import Model

from konlpy.tag import Okt
from sklearn.model_selection import train_test_split

## LSTM으로 네이버 영화 리뷰 감성 분류

In [14]:
X_train = np.load("./save/X_train.npy")
X_test = np.load("./save/X_test.npy")
y_train = np.load("./save/y_train.npy")
y_test = np.load("./save/y_test.npy")
with open('./save/vocab_size.p', 'rb') as file:
    vocab_size = pickle.load(file)

In [25]:
T = X_train.shape[1]
i = Input(shape=(T,))


x = Embedding(vocab_size, 100)(i)
x = LSTM(128)(x)
x = Dense(1, activation='sigmoid')(x)
model = Model(i, x)
model.summary()

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         [(None, 30)]              0         
_________________________________________________________________
embedding_7 (Embedding)      (None, 30, 100)           1029900   
_________________________________________________________________
lstm_6 (LSTM)                (None, 128)               117248    
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 129       
Total params: 1,147,277
Trainable params: 1,147,277
Non-trainable params: 0
_________________________________________________________________


In [7]:
X_train.shape[1]

50

In [28]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

In [29]:
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(X_train, y_train, epochs=15, callbacks=[es, mc], batch_size=60, validation_split=0.2)

Train on 19180 samples, validate on 4796 samples
Epoch 1/15
19140/19180 [============================>.] - ETA: 0s - loss: 0.7387 - acc: 0.7016
Epoch 00001: val_acc improved from -inf to 0.71685, saving model to best_model.h5
19180/19180 [==============================] - 43s 2ms/sample - loss: 0.7387 - acc: 0.7015 - val_loss: 0.6629 - val_acc: 0.7168
Epoch 2/15
19140/19180 [============================>.] - ETA: 0s - loss: 0.6502 - acc: 0.7145
Epoch 00002: val_acc improved from 0.71685 to 0.73311, saving model to best_model.h5
19180/19180 [==============================] - 38s 2ms/sample - loss: 0.6501 - acc: 0.7145 - val_loss: 0.6105 - val_acc: 0.7331
Epoch 3/15
19140/19180 [============================>.] - ETA: 0s - loss: 0.6072 - acc: 0.7261
Epoch 00003: val_acc improved from 0.73311 to 0.73895, saving model to best_model.h5
19180/19180 [==============================] - 39s 2ms/sample - loss: 0.6067 - acc: 0.7263 - val_loss: 0.5804 - val_acc: 0.7389
Epoch 4/15
19140/19180 [======

In [ ]:
loaded_model = load_model('best_model.h5')
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(X_test, y_test)[1]))